In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from datetime import timedelta
import numpy as np
from keras import Sequential
from keras.layers import SimpleRNN, Dense

ModuleNotFoundError: No module named 'tensorflow.python'

In [ ]:
df = pd.read_csv("../data/current_data.csv", sep=",")
df.drop(df.columns.difference(["date", "hour", "rented_bike_count"]), axis=1, inplace=True)

df["date"] = pd.to_datetime(df["date"] + " " + df["hour"].astype(str), format="%Y-%m-%d %H")
df.drop("hour", axis=1, inplace=True)

df.sort_values("date", inplace=True)

In [ ]:
df = df[df["date"].astype(str)]
end_date = df.iloc[df.shape[0] - 1].date.to_pydatetime() - timedelta(weeks=4)

test = []
i = 0
for x in range(df.shape[0] - 1, 0, -1):
    if df.iloc[x].date.to_pydatetime() >= end_date:
        test.append(df.iloc[x])
        i += 1
    else:
        break

train = df[0:df.shape[0] - i]
test = pd.DataFrame(test)

train = train.drop("date", axis=1)
test = test.drop("date", axis=1)

scaler = MinMaxScaler()
train[["vehicles_available"]] = scaler.fit_transform(train[["vehicles_available"]])
test[["vehicles_available"]] = scaler.transform(test[["vehicles_available"]])

In [ ]:
train_np = list(train["vehicles_available"])
windowSize, X_train, y_train = 7 * 24, [], []
for index in range(len(train) - windowSize):
    X_train.append(train_np[index:index + windowSize])
    y_train.append(train_np[index + windowSize])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train = X_train.reshape((len(X_train), 7 * 24, 1))

test_np = list(test["vehicles_available"])
X_test, y_test = [], []
for index in range(len(test) - windowSize):
    X_test.append(test_np[index:index + windowSize])
    y_test.append(test_np[index + windowSize])

X_test, y_test = np.array(X_test), np.array(y_test)

In [ ]:
EPOCH = 50

model1 = Sequential()
model1.add(SimpleRNN(16, activation="relu", input_shape=(7 * 24, 1)))
model1.add(Dense(1, activation="selu"))
model1._name = "RNN"
model1.compile(optimizer="adam", loss="mse", metrics=["mae"])

history1 = model1.fit(X_train, y_train, epochs=EPOCH)